In [1]:
import numpy as np

import pandas as pd
import time
import matplotlib.pyplot as plt
import re, string, timeit
from nltk.tag.stanford import StanfordNERTagger
import pickle
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk

In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC

In [3]:
df = pd.read_pickle('data/final_Processed_df.pkl')

In [4]:
df.head()

,authors,date,headline,link,short_description,NERtext,text,category
0,Melissa Jeltsen,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were CARDINAL Mass Shootings In GPE DATE...,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Andy McDonald,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And PERSON For WORK_OF_...,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Ron Dicker,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,PERSON For The ORDINAL Time At DATE The actor ...,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Ron Dicker,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,PERSON Blasts GPE PERSON And NORP In New Artwo...,Jim Carrey Blasts Castrato Adam Schiff And Dem...,ENTERTAINMENT
4,Ron Dicker,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",PERSON Uses PERSON Poop Bags To Pick Up After ...,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT


In [5]:
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [6]:
for i in df['text'].head():
    print(i)

There Were 2 Mass Shootings In Texas Last Week But Only 1 On TV She left her husband He killed their children Just another day in America
Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup 's Official Song Of course it has a song
Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony
Jim Carrey Blasts Castrato Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump
Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog The Dietland actress said using the bags is a really cathartic therapeutic moment


In [7]:

for i in df['NERtext'].head():
    print(i)

There Were CARDINAL Mass Shootings In GPE DATE But DATE On TV She left her husband He killed their children Just another day in GPE
Will Smith Joins Diplo And PERSON For WORK_OF_ART Of course it has a song
PERSON For The ORDINAL Time At DATE The actor and his longtime girlfriend PERSON tied the knot in a civil ceremony
PERSON Blasts GPE PERSON And NORP In New Artwork The actor gives NORP an ass-kicking for not fighting hard enough against PERSON
PERSON Uses PERSON Poop Bags To Pick Up After Her Dog The NORP actress said using the bags is a really cathartic therapeutic moment


In [8]:
###X_train, X_test, y_train, y_test  =train_test_split(df['text'], df['category'], test_size=0.2, random_state=1, stratify=df['category'])
#X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train)

In [9]:
class model():
    def __init__(self, df, textColumn = 'text', addAuthor = False, SEED = 2):
        self.SEED = SEED
        self.df = df
        self.textColumn = textColumn
        self.le = LabelEncoder()
        self.df['category_code'] = self.le.fit_transform(df['category'])
        print('-----------Sample Text---------')
        if addAuthor:
            self.df[textColumn] = self.df[textColumn] +' ' + self.df['authors']
    def lem(self, phrase):
        return " ".join([wn.lemmatize(words.lower(), pos="v") for words in word_tokenize(phrase) if (words not in stop_words_)])

    def fit(self):
        self.df[self.textColumn] = self.df[self.textColumn].apply(lambda x: self.lem(x))
        for i in self.df[self.textColumn].head(3):
            print(i)
        self.X_train, self.X_test, self.y_train, self.y_test  =train_test_split(self.df[self.textColumn], self.df['category_code'], test_size=0.2, random_state=1, stratify=self.df['category_code'])
        self.X_train, self.X_val, self.y_train, self.y_val  = train_test_split(self.X_train, self.y_train, test_size=0.25, random_state=1, stratify=self.y_train)
        vect_word = TfidfVectorizer(max_features=60000, lowercase=False, analyzer='word',tokenizer=word_tokenize,ngram_range=(1,3),dtype=np.float32)
        tr_vect = vect_word.fit_transform(self.X_train)
        vl_vect = vect_word.transform(self.X_val)
        #test_vect = vect_word.transform(X_test)
        
        #logistic regression
        self.lgr = LogisticRegression(solver='saga',multi_class='multinomial', max_iter=1000, C=1,verbose = True, n_jobs = -1, random_state = self.SEED).fit(tr_vect, self.y_train)
        self.lgr_y_pred = self.lgr.predict(vl_vect)
        print('LGR accuracy: ', (self.lgr_y_pred == self.y_val).sum()/len(self.y_val))

        #linear SVC
        
        self.svc = LinearSVC(verbose = True, random_state = self.SEED).fit(tr_vect, self.y_train)
        self.svc_y_pred = self.svc.predict(vl_vect)
        print('LinearSVC: ', (self.svc_y_pred == self.y_val).sum()/len(self.y_val))

In [10]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
there be cardinal mass shoot in gpe date but date on tv she leave husband he kill children just another day gpe melissa jeltsen
will smith join diplo and person for work_of_art of course song andy mcdonald
person for the ordinal time at date the actor longtime girlfriend person tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 22 epochs took 34 seconds
LGR accuracy: 

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   34.0s finished


 0.7082721366159668
[LibLinear]LinearSVC:  0.7256229618381419


In [11]:
textColumnToUse = 'text'
model = model(df, textColumnToUse, addAuthor)
model.fit()

-----------Sample Text---------
there be 2 mass shoot in texas last week but only 1 on tv she leave husband he kill children just another day america melissa jeltsen
will smith join diplo and nicky jam for the 2018 world cup 's official song of course song andy mcdonald
hugh grant marry for the first time at age 57 the actor longtime girlfriend anna eberstein tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 19 epochs took 29 seconds
LGR accuracy:  0.7315974210251176


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   29.1s finished


[LibLinear]LinearSVC:  0.7522839859600209


In [52]:
#no tunning
#Logistic: vectorize, remove stopwords 0.560752781857559
#Logistic: both headline and shortdescription, vectorize, remove stopwords 0.5953797515620721

#LinearSVC: both headline and shortdescription, vectorize, remove stopwords 0.6011799556894277

In [53]:
# without author NER LGR accuracy:  0.5542804510716686 LinearSVC:  0.554429812551343
# without author LGR accuracy:  0.6009310198899703 LinearSVC:  0.6112618555674492

# with Author NER LGR accuracy:  0.7082721366159668 LinearSVC:   0.7256229618381419
# with Author LGR accuracy:  0.7315974210251176 LinearSVC:  0.7522839859600209
